# <center>Implementing LDA in Python</center>

<center>Dr. W.J.B. Mattingly</center>

<center>Smithsonian Data Science Lab and United States Holocaust Memorial Museum</center>

<center>February 2021</center>

## Key Concepts in this Notebook

## Introduction

## Importing the Required Libraries

In [56]:
#https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/#1introduction
import numpy as np
import pandas as pd
import json
import glob

#Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

#spacy
import spacy
from nltk.corpus import stopwords

#vis
import pyLDAvis
# import pyLDAvis.gensim
import pyLDAvis.gensim_models as gensimvis

# Translator
from googletrans import Translator
translator = Translator()

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

## Preparing the Data

In [57]:
def load_data(file):
    with open (file, "r", encoding="utf-8") as f:
        data = json.load(f) 
    return (data)

def write_data(file, data):
    with open (file, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=4)


In [58]:
stopwords = stopwords.words("english")

In [64]:
# data = load_data("data/ushmm_dn.json")["texts"]

url = '/Users/dj/Documents/GitHub/out.csv'
df = pd.read_csv(url)

# str(translator.translate(df.iloc[i,0], dest = 'en', src = 'nl'))

data = [df.iloc[i,0] for i in np.arange(100)]

print (data[0][0:50])

Onderzoek van de zaakDit arrest is gewezen naar aa


In [9]:
url = '/Users/dj/Documents/GitHub/inbraak_'
numbers = np.arange(1,7)
ending = '.txt'

texts = []
for i in numbers:
    p = url + str(i) + ending
    f = open(p,'r')
    text = f.read()
    texts.append(text)

data = texts

In [65]:
def lemmatization(texts, allowed_postags=["NOUN", "ADJ", "VERB", "ADV"]):
    nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])
    texts_out = []
    for text in texts:
        doc = nlp(text)
        new_text = []
        for token in doc:
            if token.pos_ in allowed_postags:
                new_text.append(token.lemma_)
        final = " ".join(new_text)
        texts_out.append(final)
    return (texts_out)


lemmatized_texts = lemmatization(data)
print (lemmatized_texts[0][0:90])

arrest gewezen aanleide onderzoek ter terechtzitte hoger voormeld het hof heeft kennisgeno


In [66]:
def gen_words(texts):
    final = []
    for text in texts:
        new = gensim.utils.simple_preprocess(text, deacc=True)
        final.append(new)
    return (final)

data_words = gen_words(lemmatized_texts)

print (data_words[0][0:20])

['arrest', 'gewezen', 'aanleide', 'onderzoek', 'ter', 'terechtzitte', 'hoger', 'voormeld', 'het', 'hof', 'heeft', 'kennisgenoman', 'vordere', 'generaal', 'raadsman', 'gebracht']


In [67]:
id2word = corpora.Dictionary(data_words)

corpus = []
for text in data_words:
    new = id2word.doc2bow(text)
    corpus.append(new)

print (corpus[0][0:20])

word = id2word[[0][:1][0]]
print (word)

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1)]
aanleide


In [68]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=3,
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha="auto")


## Vizualizing the Data

In [69]:
pyLDAvis.enable_notebook()
vis = gensimvis.prepare(lda_model, corpus, id2word, mds="mmds", R=30)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.016995  0.071953       1        1  52.037085
0      0.068344 -0.082237       2        1  26.372386
1     -0.085339  0.010284       3        1  21.590529, topic_info=                Term        Freq       Total Category  logprob  loglift
13      terechtzitte   53.000000   53.000000  Default  30.0000  30.0000
367      slachtoffer   59.000000   59.000000  Default  29.0000  29.0000
282        verklaart   26.000000   26.000000  Default  28.0000  28.0000
4            gewezen   21.000000   21.000000  Default  27.0000  27.0000
6                het  479.000000  479.000000  Default  26.0000  26.0000
..               ...         ...         ...      ...      ...      ...
8              hoger    9.489041   63.320451   Topic3  -5.2458  -0.3652
12               ter    7.908413   47.574670   Topic3  -5.4280  -0.2615
154           andere    7.309487   37.820264   Topic3  -5.5067  -0.1108
192  gevangenisstraf    6.541207   40.662128   Topic3  -5.6178  -0.2943
173             acht    5.710395   16.797429   Topic3  -5.7536   0.4540

[225 rows x 6 columns], token_table=      Topic      Freq          Term
term                               
1377      2  0.879179  aangetroffen
744       1  0.632082      aangever
744       2  0.035116      aangever
744       3  0.316041      aangever
0         1  0.135706      aanleide
...     ...       ...           ...
285       2  0.877285         zitte
470       2  0.820003       zodanig
286       1  0.101652         zover
286       2  0.813218         zover
147       1  0.951553        zullen

[289 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 1, 2])

In [2]:
# Import Mannes Data
import pandas as pd

In [12]:
url = '/Users/dj/Documents/GitHub/out.csv'
df = pd.read_csv(url)
data = []
for i in np.arange(df.shape[0]):
    data.append(df.iloc[i,0])

In [13]:
data = [df.iloc[i,0] for i in np.arange(df.shape[0])]

In [15]:
data[0]

'Onderzoek van de zaakDit arrest is gewezen naar aanleiding van het onderzoek ter terechtzitting in hoger beroep van 20 januari 2021.Tegen voormeld vonnis is namens de verdachte hoger beroep ingesteld. Het hof heeft kennisgenomen van de vordering van de advocaat-generaal en van hetgeen door de raadsman naar voren is gebracht.'